In [68]:
import pandas as pd
import json
from scipy import stats
import numpy as np
import re
import csv

In [2]:
## keyword
read_path = '../csv/search_click_200419_200425.csv'
raw_keyword = pd.read_csv(read_path)
raw_keyword.head()

,ref_term,name,click
0,닌텐도 스위치,디지털/가전,401150
1,아이폰,디지털/가전,130835
2,노트북,디지털/가전,125017
3,에어팟,디지털/가전,115235
4,닌텐도 스위치 라이트,디지털/가전,108723


In [3]:
raw_keyword['click'].sum()

48762211

In [4]:
raw_keyword.groupby(['name'], as_index=False).agg({'click': 'sum'})

,name,click
0,(구)패션의류,3
1,구인구직,38434
2,기타,1380744
3,남성의류,6122766
4,도서/티켓/취미/애완,3223692
5,디지털/가전,10842509
6,번개나눔,84139
7,뷰티/미용,1461129
8,생활/문구/가구/식품,2188223
9,스타굿즈,2885185


In [5]:
digital = raw_keyword[raw_keyword['name'] == '디지털/가전']
digital_1000 = digital.head(1000)
# save
save_path = '../csv/result/digital_1000.csv'
digital_1000.to_csv(save_path, index=False, mode='w', header=True)

In [56]:
## keyword
read_path = '../csv/digital_1000_naver_scrap_result.csv'
digital_naver = pd.read_csv(read_path)
digital_naver = digital_naver[(digital_naver['filter_value'] != '-') & 
                              (digital_naver['filter_value'] != '정확한 검색어 인지 확인하시고 다시 검색해 주세요.')] # 결과 없는 row 제거
digital_naver = digital_naver[(digital_naver['filter_key'] != '가격') &
                              (digital_naver['filter_key'] != '배송/혜택/색상') & 
                              (digital_naver['filter_key'] != '쇼핑연관')] # 가격, 배송/혜택/색상, 쇼핑연관 key 제거
digital_naver.tail()

,original_keyword,keyword_query,filter_key,filter_value,search_count
6117,lg g6,lg+g6,내장메모리,"['128GB', '4GB', '32GB', '64GB']",1530
6121,마리오파티,마리오파티,카테고리,"['게임타이틀', '휴대용게임기']",1336
6122,마리오파티,마리오파티,브랜드,"['닌텐도', '노브랜드', '파티해', '아트박스', 'OST', '바보사랑', ...",1336
6126,gtx1070ti,gtx1070ti,카테고리,"['NVIDIA계열', 'AMD계열']",1335
6127,gtx1070ti,gtx1070ti,브랜드,"['MSI', '조텍', 'ASUS', '기가바이트']",1335


In [62]:
filter_keys =[]
def get_filter_key(row):
    key = row['filter_key']
    if key not in filter_keys:
        filter_keys.append(key)
    return None

digital_naver[['filter_key']].apply(get_filter_key, axis=1)

filters = {key: [] for key in filter_keys}

def get_filter_values(row):
    key = row['filter_key']
    values = row['filter_value'].strip("[").strip("]").replace("'","").split(", ")
    
    filters_value = filters[key]
    
    for value in values:
        if value not in filters_value:
            filters[key].append(value)
    return None

digital_naver[['filter_key', 'filter_value']].apply(get_filter_values, axis=1)

1       None
2       None
3       None
7       None
8       None
        ... 
6117    None
6121    None
6122    None
6126    None
6127    None
Length: 3219, dtype: object

In [67]:
pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in filters.items() ]))

,카테고리,시리즈,게임장르,화면크기,키워드추천,제조사,영상출력,부가기능,CPU,배터리 타입,...,세탁방법,주요소재,구성품(카드),적용모델,상판소재,탈취기능,적외선램프,안마부위,구조,타입
0,휴대용게임기,WII,어드벤처,~4인치,맥스,LG전자,miniHDMI,터치펜,코어i5,리튬이온,...,세탁기가능,인견,카드리더포함,닌텐도스위치,폴리우드,"냄새,연기제거",높이조절,허리,냉장1칸+냉동1칸,양문형
1,게임타이틀,3DS,액션어드벤처,~5인치,프로,삼성전자,HDMI,터치펜(별매),라이젠5-3500U,리튬폴리머,...,드라이클리닝,면,어댑터포함,"스마트폰,태블릿PC용",원목,"냄새,연기제거(3단계)",좌우조절,등,냉장2칸+냉동2칸,테이블형
2,SKT,2DS,추리 어드벤처,~5.6인치,프로맥스,ASUS,다중모니터연결(3개),터치펜(와콤),라이젠3-3200U,NaN,...,NaN,모달,어댑터미포함,맥북프로레티나15인치,스테인리스,NaN,NaN,손,냉장4칸+냉동2칸,사각타입
3,자급제폰,DS,스포츠,5.7인치~,기기변경,레노버,WiDi(무선연결),웹캠,라이젠7-3700U,NaN,...,NaN,폴리에스테르,NaN,NaN,알루미늄,NaN,NaN,눈,냉장3칸+냉동1칸,조립형
4,KT,아이폰8,파티,6인치~,11프로맥스,HP,다중모니터연결(2개),S펜,코어i5-10210U,NaN,...,NaN,NaN,NaN,NaN,메시,NaN,NaN,전신,냉장2칸+냉동1칸,곡면타입
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2768,NaN,NaN,NaN,NaN,2020북미판,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2769,NaN,NaN,NaN,NaN,키즈,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2770,NaN,NaN,NaN,NaN,ps3저스트댄스,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2771,NaN,NaN,NaN,NaN,3wii,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
file = open('../csv/result/digital_1000_named_entity.csv', mode="w")
writer = csv.writer(file)
writer.writerow(["named_entity", "name"])

for key in filters.keys():
    result =[]
    for column in filters[key]:
        result = [key, column]
        writer.writerow(result)